In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go

from datetime import datetime

In [2]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')


# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [60]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

## Assumptions

In [73]:
cpi = 0.028
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_litres_per_100km = 7.0

purchase_price = 35000
age_at_purchase = 0
years_to_model = 15

## Standing Costs

In [74]:
insurance_per_year = 500
registration_per_year = 500
service_cost = 400
service_interval_km = 10000
service_interval_years = 1.0
roadside_assist_per_year = 200

## Depreciation

In [75]:
def depreciation_rate_for_age(age_in_years):
    '''Go with the simple assumption of 15% for first 3 years, and 10% after that.
       This might be too aggressive, but can fine-tune later.
    '''
    if age_in_years < 3:
        return 0.15
    else:
        return 0.10

### Calculate Depreciation

In [76]:
dep_value = np.zeros(years_to_model)
yearly_depreciation_cost = np.zeros(years_to_model)
dep_value[0] = purchase_price
previous = dep_value[0]
for yr in range(1, years_to_model):
    previous *= (1.0 - depreciation_rate_for_age(age_at_purchase + yr))
    dep_value[yr] = previous
    yearly_depreciation_cost[yr-1] = dep_value[yr-1] - dep_value[yr]

`yearly_depreciation_cost` is the depreciation loss for the given year, defined as the difference in value between the start and end of the year. To calculate the final year loss, we therefore need to calculate the depreciated value one year past the range defined by `years_to_model`

In [77]:
yearly_depreciation_cost[-1] = dep_value[-1] * depreciation_rate_for_age(age_at_purchase + years_to_model)

In [78]:
line_chart(dep_value)

In [79]:
line_chart(yearly_depreciation_cost)

# Experimental Code

# Compound interest formula
Solves $A = P(1+\frac{r}{n})^{nt}$ for
* P = principal
* r = annual rate
* n = number of times interest is compounded per year
* t = number of years to compound

Returns an array giving the value for each year in the sequence.

In [83]:
def compound_interest(
    principal,
    annual_rate,
    years,
    compounds_per_year=1,
):
    return np.array([principal * ((1 + annual_rate/compounds_per_year)**(yr*compounds_per_year)) for yr in range(years)])

In [84]:
indexed_fuel_cost = compound_interest(fuel_cost_per_litre, cpi, years_to_model)
                                      #np.array([fuel_cost_per_litre * (1 + cpi)**yr for yr in range(years_to_model)])
indexed_fuel_cost

array([1.5       , 1.542     , 1.585176  , 1.62956093, 1.67518863,
       1.72209392, 1.77031255, 1.8198813 , 1.87083797, 1.92322144,
       1.97707164, 2.03242964, 2.08933767, 2.14783913, 2.20797862])

In [85]:
assert len(indexed_fuel_cost) == years_to_model
assert len(indexed_fuel_cost) == len(dep_value)
assert len(indexed_fuel_cost) == len(yearly_depreciation_cost)